In [ ]:
!git clone https://{GITHUB_ACCESS_TOKEN}@github.com/AliMohseninejad/ganbert-classifier.git
!rm ganbert-classifier/Codes/main.ipynb
!cp -r ganbert-classifier/Codes/data/ ./
!cp -r ganbert-classifier/Codes/evaluation/ ./
!cp -r ganbert-classifier/Codes/model/ ./
!cp -r ganbert-classifier/Codes/training/ ./
!cp -r ganbert-classifier/Dataset/ ../
!cp -r ganbert-classifier/Plots/ ../